In [ ]:
!pip install flask --break-system-packages
!pip install flask-restx  --break-system-packages

In [ ]:
import nest_asyncio
from flask import Flask, jsonify, request
from flask_restx import Api, Resource, Namespace
from werkzeug.serving import run_simple

    # Allow Flask to run in Jupyter
nest_asyncio.apply()

app = Flask(__name__)

api = Api(app, 
          title='Message Example API with Path Variables and Parameters', 
          version='1.0', 
          description='An example API demonstrating CRUD operations, path variables, and query parameters.')

# Create a Namespace for the API
message_example_ns = Namespace('messages', description='Message Operations with Path Variables and Query Parameters')

@message_example_ns.route('/<string:name>') # for instance name = adam => localhost:5000/adam
class GreetMessage(Resource):
    def get(self, name):
        """
        A simple API to greet a user by name (path variable).
        """
        return {"message": f"Hello, {name}!"}, 200

@message_example_ns.route('/query')
class QueryMessage(Resource):
    def get(self):
        """
        A simple API to demonstrate query parameters.
        """
        name = request.args.get('name', 'Guest')
        age = request.args.get('age')
        if age:
            return {"message": f"Hello, {name}! You are {age} years old."}, 200
        return {"message": f"Hello, {name}!"}, 200

# Add the Namespace to the API
api.add_namespace(message_example_ns, path='/messages')

# Run the app in Jupyter
run_simple('localhost', 5000, app)


In [ ]:
import nest_asyncio
from flask import Flask, jsonify, redirect, url_for
from flask_restx import Api, Resource, Namespace
from werkzeug.serving import run_simple

# Allow Flask to run in Jupyter
nest_asyncio.apply()

app = Flask(__name__)
api = Api(app, 
          title='Hello Goodbye API', 
          version='1.0', 
          description='An API with Hello, Goodbye, and Redirect endpoints using Flask-RESTx.')

# Create a Namespace for the API
# A Namespace in Flask-RESTx is a way to group and organize related API endpoints.
# It allows you to define specific routes and resources under a common namespace, 
# making the API more modular and easier to maintain.
# greetings Hello and Goodbye operations
hello_ns = Namespace('greetings', description='Hello and Goodbye operations')
#  hello_ns = {}
""" 
# hello_ns["hello_endpoint"] = "localhost:5000/hello" 
# ::: hello_ns={ "hello_endpoint": "localhost:5000/hello"}
"""
@hello_ns.route('/hello', endpoint="hello_endpoint") # name of endpoint = hello_endpoint, endpoint=/hello address=localhost:5000/api/hello
class HelloWorld(Resource): #hello_ns
    def get(self):
        """
        A simple Hello World API.
        """
        return {"message": "Hello, World!"}

""" 
# hello_ns["goodbye_endpoint"] = "localhost:5000/goodbye" 
# ::: hello_ns={"hello_endpoint": "localhost:5000/hello", "goodbye_endpoint": "localhost:5000/goodbye" }
"""
@hello_ns.route('/goodbye', endpoint="goodbye_endpoint")   # name of endpoint = goodbye_endpoint, endpoint=/hello address=localhost:5000/api/goodbye
class GoodbyeWorld(Resource):
    def get(self):
        """
        A simple Goodbye World API.
        """
        return {"message": "Goodbye, World!"}

""" 
# hello_ns["redirect_to_hello_endpoint"] = "localhost:5000/redirect" :::
# hello_ns={"hello_endpoint": "localhost:5000/hello", "goodbye_endpoint": "localhost:5000/goodbye", "redirect_to_hello_endpoint": "localhost:5000/redirect"}
"""
@hello_ns.route('/redirect', endpoint="redirect_to_hello_endpoint") #"localhost:5000/goodbye", endpoint=/hello address=localhost:5000/api/redirect => localhost:5000/api/hello
class RedirectToHello(Resource):
    def get(self):
        """
        Redirect to the Hello World endpoint.
        def url_for(hello_endpoint: str) -> str:
            return hello_ns[hello_endpoint]
            
        """
        return redirect(url_for('hello_endpoint')) 

# Add the Namespace to the API
api.add_namespace(hello_ns, path='/api')

# Run the app in Jupyter
run_simple('localhost', 5000, app)


In [9]:
import nest_asyncio
from flask import Flask, request
from flask_restx import Api, Resource, Namespace
from flask import render_template_string
from werkzeug.serving import run_simple

# Allow Flask to run in Jupyter
nest_asyncio.apply()

# Initialize Flask app
app = Flask(__name__)
api = Api(app, 
          title='Template Example API', 
          version='1.0', 
          description='An example API demonstrating templates with Flask-RESTx.')

# Templates stored as strings
templates = {
    "greet.html": """  
    <h1>Hello, {{ name }}! </h1>
    """,
    "query.html": """
    <h1>  Hello, {{ name }}! {% if age %}You are {{ age }} years old.{% else %}Age not provided.{% endif %} /h1>
    """
}

# Create Namespace for API
template_example_ns = Namespace('templates', description='Operations with templates and RESTx')

@template_example_ns.route('/greet/<string:name>')
class GreetMessage(Resource):
    def get(self, name):
        """
        A simple API to greet a user by name (path variable).
        Renders a template with the greeting message.
        """
        return render_template_string(templates['greet.html'], name=name)

@template_example_ns.route('/query')
class QueryMessage(Resource):
    def get(self):
        """
        A simple API to demonstrate query parameters.
        Renders a template with the greeting message and optional age.
        """
        name = request.args.get('name', 'Guest')
        age = request.args.get('age')
        return render_template_string(templates['query.html'], name=name, age=age)

# Add the Namespace to the API
api.add_namespace(template_example_ns, path='/templates')

# Run the app in Jupyter
run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2024 15:28:07] "GET /templates/greet/adam HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2024 15:28:07] "GET /templates/query?name=Awet&age=89 HTTP/1.1" 200 -


In [ ]:
import nest_asyncio
from flask import Flask, request
from flask_restx import Api, Resource, Namespace
from werkzeug.serving import run_simple

# Allow Flask to run in Jupyter
nest_asyncio.apply()

app = Flask(__name__)
api = Api(app, 
          title='CRUD Example API', 
          version='1.0', 
          description='An example API demonstrating POST, PUT, and DELETE operations with Flask-RESTx.')

# Create a Namespace for the API
example_ns = Namespace('example', description='CRUD Operations')

# Create a dictionary to store messages (simulating a database)
messages_db = {}

""" 
example_ns = {
    "messages_endpoint": {
        "url": "localhost:5000/example/message"
        "methods": {
            "POST": post,
            "PUT": put,
            "DELETE": delete,
        } 
    }
    }

"""
@example_ns.route('/message', endpoint="messages_endpoint") # 
class Message(Resource):
    def post(self):
        """
        Create a new message.
        """
        data = request.get_json()
        if not data or 'id' not in data or 'message' not in data:
            return {"error": "ID and Message are required"}, 400
        
        # Check if the ID already exists
        if data['id'] in messages_db:
            return {"error": "Message with this ID already exists"}, 400
        
        # Add the new message
        messages_db[data['id']] = data['message']
        return {"message": f"Created: {data['message']}"}, 201

    def put(self):
        """
        Update an existing message.
        """
        data = request.get_json()
        if not data or 'id' not in data or 'message' not in data:
            return {"error": "ID and Message are required"}, 400
        
        # Check if the ID exists
        if data['id'] not in messages_db:
            return {"error": "Message with this ID does not exist"}, 404
        
        # Update the message
        messages_db[data['id']] = data['message']
        return {"message": f"Updated: {data['message']}"}, 200

    def delete(self):
        """
        Delete an existing message.
        """
        data = request.get_json()
        if not data or 'id' not in data:
            return {"error": "ID is required"}, 400
        
        # Check if the ID exists
        if data['id'] not in messages_db:
            return {"error": "Message with this ID does not exist"}, 404
        
        # Delete the message
        deleted_message = messages_db.pop(data['id'])
        return {"message": f"Deleted: {deleted_message}"}, 200

# Add the Namespace to the API
api.add_namespace(example_ns, path='/example')

# Run the app in Jupyter
run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2024 15:29:17] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2024 15:30:07] "POST /example/message HTTP/1.1" 415 -
127.0.0.1 - - [22/Dec/2024 15:30:26] "POST /example/message HTTP/1.1" 415 -
127.0.0.1 - - [22/Dec/2024 15:31:57] "POST /example HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2024 15:32:53] "POST /example//message HTTP/1.1" 308 -
127.0.0.1 - - [22/Dec/2024 15:32:53] "POST /example/message HTTP/1.1